# Aiida

## Define workflow with aiida

In [ ]:
from python_workflow_definition.aiida import AiidaPwdConverter

from aiida_workgraph import WorkGraph, task
from aiida import orm, load_profile
from rich.pretty import pprint

load_profile()

workflow_json_filename =  "aiida_simple.json"

In [8]:
from workflow import (
    get_sum as _get_sum,
    get_prod_and_div as _get_prod_and_div,
    square as _square
)

In [9]:
wg = WorkGraph("arithmetic")

In [10]:
get_prod_and_div_task = wg.add_task(
    task(outputs=['prod', 'div'])(_get_prod_and_div),
    name="get_prod_and_div",
    x=orm.Float(1),
    y=orm.Float(2),
)

In [11]:
get_sum_task = wg.add_task(
    _get_sum,
    name="get_sum",
    x=get_prod_and_div_task.outputs.prod,
    y=get_prod_and_div_task.outputs.div,
)

In [12]:
square_task = wg.add_task(
    _square,
    name="square",
    x=get_sum_task.outputs.result,
)

In [ ]:
aiida_converter = AiidaPwdConverter()
model = aiida_converter.workgraph_to_model(wg=wg)
pprint(model.model_dump())

model.dump_json_file(file_name=workflow_json_filename)

{
│   'nodes': [
│   │   {'id': 0, 'type': 'function', 'value': 'workflow.get_prod_and_div', 'name': 'get_prod_and_div'},
│   │   {'id': 1, 'type': 'function', 'value': 'workflow.get_sum', 'name': 'get_sum'},
│   │   {'id': 2, 'type': 'function', 'value': 'workflow.square', 'name': 'square'},
│   │   {'id': 3, 'type': 'input', 'name': 'x', 'value': 1.0},
│   │   {'id': 4, 'type': 'input', 'name': 'y', 'value': 2.0},
│   │   {'id': 5, 'type': 'output', 'name': 'result', 'value': None}
│   ],
│   'edges': [
│   │   {'target': 0, 'targetPort': 'x', 'source': 3, 'sourcePort': None},
│   │   {'target': 0, 'targetPort': 'y', 'source': 4, 'sourcePort': None},
│   │   {'target': 1, 'targetPort': 'x', 'source': 0, 'sourcePort': 'prod'},
│   │   {'target': 1, 'targetPort': 'y', 'source': 0, 'sourcePort': 'div'},
│   │   {'target': 2, 'targetPort': 'x', 'source': 1, 'sourcePort': None},
│   │   {'target': 5, 'targetPort': None, 'source': 2, 'sourcePort': None}
│   ]
}

In [15]:
# Currently conversion PWD <-> WG is not atomic, as information is lost due to the `group_outputs` workaround, and the
# fact that `group_inputs` is not implemented in WG, at all

# pwd_wf = PwdWorkflow.load_json_file(workflow_json_filename)
# wg = aiida_converter.model_to_workgraph(workflow_model=pwd_wf)

# model = aiida_converter.workgraph_to_model(wg)
# pprint(model.model_dump())

In [ ]:
!cat {workflow_json_filename}

## Load Workflow with jobflow

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

In [ ]:
from jobflow.managers.local import run_locally

In [ ]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
result = run_locally(flow)
result

## Load Workflow with pyiron_base

In [ ]:
from python_workflow_definition.pyiron_base import load_workflow_json

In [ ]:
delayed_object_lst = load_workflow_json(file_name=workflow_json_filename)
delayed_object_lst[-1].draw()

In [ ]:
delayed_object_lst[-1].pull()